In [1]:
from luxai_s2.env import LuxAI_S2
from luxai_s2.map.board import Board
from luxai_s2.map_generator.generator import GameMap
import matplotlib.pyplot as plt
import matplotlib
import logging
import numpy as np
from luxai_s2.utils import my_turn_to_place_factory
from agents_experiments import (reset_w_custom_board, IdleAgent, ControlledAgent, interact)
from plots import plot_factory_stats
import uuid
from IPython.display import Video
from pprint import pprint

import pandas as pd
# import seaborn as sns

matplotlib.rc('font', size=14)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
# key-values of dict below should match attributes present in this class
# https://github.com/aernesto/Lux-Design-S2/blob/adrian/luxai_s2/luxai_s2/config.py#L34
environment_config = dict(
#         max_episode_length=350,  # how many total turns in game
#         CYCLE_LENGTH=50,
#         DAY_LENGTH=50,  # so no night
        MAX_RUBBLE=0,  # disable rubble   
        MIN_FACTORIES=2,  
        MAX_FACTORIES=2
    )
env = LuxAI_S2(**environment_config)
custom_rubble = np.zeros((env.env_cfg.map_size,) * 2, dtype=np.int64)

# Let's put a single ice at 30,30
custom_ice = np.zeros_like(custom_rubble)
custom_ice[30, 30] = 1

# Let's put a single ore at 21, 21
custom_ore = np.zeros_like(custom_rubble)
custom_ore[21, 21] = 1

custom_symmetry = None


custom_map_ = GameMap(custom_rubble, custom_ice, custom_ore, custom_symmetry)

simple_board = Board(
    seed=env.seed_rng.randint(0, 2**32 - 1, dtype=np.int64), 
    env_cfg=env.env_cfg,
    existing_map=custom_map_
)

obs, env = reset_w_custom_board(env, seed=41, custom_board=simple_board)
    
# source code of reset() method: 
# https://github.com/aernesto/Lux-Design-S2/blob/adrian/luxai_s2/luxai_s2/env.py#L175
# obs = env.reset(seed=41) # resets an environment with a seed

In [3]:
# first player cannot place factory on any boundary, so inner square has dimensions 46x46
# the 2 isolated resources remove each 9 tiles from the potential spawn locations
46*46- 9*2

2098

In [4]:
2098 -28

2070

In [5]:
logger.setLevel(logging.DEBUG)

filename = str(uuid.uuid4()) + '.mp4'
file_path = '/home/adrian_admin/prog/kaggle/lux-ai-2/Lux-Design-S2/data/experiments/adrian/'
animate_arg = ''  # file_path + filename

agents = {}
for player, agent_constructor in zip(env.agents, [IdleAgent, ControlledAgent]):
    logging.debug(player)
    agents[player] = agent_constructor(player, env.state.env_cfg) 
    
_ = interact(
    env, 
    agents, 
    30, 
    animate_=animate_arg, 
    break_at_first_action=False, 
    debug=False, 
    custom_board=simple_board
)
# print(last_obs)

DEBUG:root:player_0
DEBUG:root:player_1


In [6]:
if animate_arg:
    Video(animate_arg, embed=True)

In [7]:
# #video above is missing bottom pixels for some reason 
# logger.setLevel(logging.INFO)
# img = env.render("rgb_array", width=640, height=640)
# f, ax = plt.subplots(figsize=(7, 7))
# ax.imshow(img);

In [8]:
# plot_factory_stats(agents['player_1'].stats);

# Network

In [9]:
import itertools
import networkx as nx

In [10]:
G = nx.MultiDiGraph()
board_length = 3  # env.env_cfg.map_size
toy_rubble=np.array([[1, 1, 2], [0, 2, 3], [0, 0, 0]])
G.add_nodes_from(itertools.product(range(board_length), range(board_length)))

In [11]:
edges = []
# x goes left to right
# y goes top to bottom
for x, y in itertools.product(range(board_length), range(board_length)):
    weight = toy_rubble.T[x, y]
    at_top_edge = y == 0
    at_bottom_edge = y == board_length - 1
    at_left_edge = x == 0
    at_right_edge = x == board_length - 1
    if not at_top_edge:
        edges.append(((x, y - 1), (x, y), dict(weight=weight)))
    if not at_bottom_edge:
        edges.append(((x, y + 1), (x, y), dict(weight=weight)))
    if not at_left_edge:
        edges.append(((x - 1, y), (x, y), dict(weight=weight)))
    if not at_right_edge:
        edges.append(((x + 1, y), (x, y), dict(weight=weight)))
G.add_edges_from(edges)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
G.graph

{}

In [13]:
edges

[((0, 1), (0, 0), {'weight': 1}),
 ((1, 0), (0, 0), {'weight': 1}),
 ((0, 0), (0, 1), {'weight': 0}),
 ((0, 2), (0, 1), {'weight': 0}),
 ((1, 1), (0, 1), {'weight': 0}),
 ((0, 1), (0, 2), {'weight': 0}),
 ((1, 2), (0, 2), {'weight': 0}),
 ((1, 1), (1, 0), {'weight': 1}),
 ((0, 0), (1, 0), {'weight': 1}),
 ((2, 0), (1, 0), {'weight': 1}),
 ((1, 0), (1, 1), {'weight': 2}),
 ((1, 2), (1, 1), {'weight': 2}),
 ((0, 1), (1, 1), {'weight': 2}),
 ((2, 1), (1, 1), {'weight': 2}),
 ((1, 1), (1, 2), {'weight': 0}),
 ((0, 2), (1, 2), {'weight': 0}),
 ((2, 2), (1, 2), {'weight': 0}),
 ((2, 1), (2, 0), {'weight': 2}),
 ((1, 0), (2, 0), {'weight': 2}),
 ((2, 0), (2, 1), {'weight': 3}),
 ((2, 2), (2, 1), {'weight': 3}),
 ((1, 1), (2, 1), {'weight': 3}),
 ((2, 1), (2, 2), {'weight': 0}),
 ((1, 2), (2, 2), {'weight': 0})]

In [14]:
# logger.setLevel(logging.INFO)
# nx.draw(G)

In [15]:
list(G.nodes)

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [16]:
G.edges.data()

OutMultiEdgeDataView([((0, 0), (0, 1), {'weight': 0}), ((0, 0), (1, 0), {'weight': 1}), ((0, 1), (0, 0), {'weight': 1}), ((0, 1), (0, 2), {'weight': 0}), ((0, 1), (1, 1), {'weight': 2}), ((0, 2), (0, 1), {'weight': 0}), ((0, 2), (1, 2), {'weight': 0}), ((1, 0), (0, 0), {'weight': 1}), ((1, 0), (1, 1), {'weight': 2}), ((1, 0), (2, 0), {'weight': 2}), ((1, 1), (0, 1), {'weight': 0}), ((1, 1), (1, 0), {'weight': 1}), ((1, 1), (1, 2), {'weight': 0}), ((1, 1), (2, 1), {'weight': 3}), ((1, 2), (0, 2), {'weight': 0}), ((1, 2), (1, 1), {'weight': 2}), ((1, 2), (2, 2), {'weight': 0}), ((2, 0), (1, 0), {'weight': 1}), ((2, 0), (2, 1), {'weight': 3}), ((2, 1), (1, 1), {'weight': 2}), ((2, 1), (2, 0), {'weight': 2}), ((2, 1), (2, 2), {'weight': 0}), ((2, 2), (1, 2), {'weight': 0}), ((2, 2), (2, 1), {'weight': 3})])

In [17]:
print(nx.shortest_path(G, source=(0, 0), target=(2, 2), weight='weight'))

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]


In [18]:
toy_rubble

array([[1, 1, 2],
       [0, 2, 3],
       [0, 0, 0]])

In [19]:
print(nx.shortest_path(G, source=(2, 1), target=(0, 0), weight='weight'))

[(2, 1), (2, 2), (1, 2), (0, 2), (0, 1), (0, 0)]
